# Load Data
kaggle.com/anirudhg15/mammals-classification

In [ ]:
#import library 
import pandas as pd
import numpy as np
import os, cv2, shutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm as tq
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, GlobalMaxPooling2D, GlobalAveragePooling2D, AveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from google.colab import files
from keras.preprocessing import image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Kaggle/

In [ ]:
file = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
api_token = {"username":"dinaardilia","key":"78757312cf856b19fb52c0aeb892c15a"}
import json
import zipfile
import os
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
%cd /content/

/content


In [ ]:
! kaggle datasets download -d anirudhg15/mammals-classification

 98% 499M/508M [00:04<00:00, 97.3MB/s]
100% 508M/508M [00:04<00:00, 113MB/s] 


In [ ]:
ls

mammals-classification.zip  sample_data/


In [ ]:
# extract data zip
! unzip mammals-classification.zip

Archive:  mammals-classification.zip
  inflating: mammals/bear/00000000.jpg  
  inflating: mammals/bear/00000001.jpg  
  inflating: mammals/bear/00000002.JPG  
  inflating: mammals/bear/00000003.jpg  
  inflating: mammals/bear/00000004.JPG  
  inflating: mammals/bear/00000005.jpg  
  inflating: mammals/bear/00000006.jpg  
  inflating: mammals/bear/00000007.jpg  
  inflating: mammals/bear/00000008.jpg  
  inflating: mammals/bear/00000009.jpg  
  inflating: mammals/bear/00000010.jpg  
  inflating: mammals/bear/00000011.jpg  
  inflating: mammals/bear/00000012.JPG  
  inflating: mammals/bear/00000013.jpg  
  inflating: mammals/bear/00000014.jpg  
  inflating: mammals/bear/00000015.jpg  
  inflating: mammals/bear/00000016.jpg  
  inflating: mammals/bear/00000017.jpg  
  inflating: mammals/bear/00000018.jpg  
  inflating: mammals/bear/00000019.jpg  
  inflating: mammals/bear/00000020.jpg  
  inflating: mammals/bear/00000021.jpg  
  inflating: mammals/bear/00000022.jpg  
  inflating: mammals

In [ ]:
ls

mammals/  mammals-classification.zip  sample_data/


In [ ]:
%cd /content/mammals/

/content/mammals


In [ ]:
os.listdir('/content/mammals/')

['horse', 'elephant', 'cat', 'dog', 'tiger', 'wolf', 'bear', 'lion', 'goat']

In [ ]:
from PIL import Image
base_dataset = '/content/mammals/'
class_dir = ['bear', 'cat', 'dog', 'elephant', 'goat', 'horse', 'lion', 'tiger', 'wolf']
for class_item in class_dir:
  cur_dir = base_dataset+"/"+class_item
  dataset = os.listdir(cur_dir)
  for item in dataset:
    if not item.endswith(".jpg"):
        os.remove(os.path.join(cur_dir, item))

In [ ]:
file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(base_dataset):
    for name in files:
        full_path.append(os.path.join(path, name)) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

In [ ]:
# membuat dataframe dari variabel baru
data = pd.DataFrame({'path':full_path,'file_name':file_name,'tag':tag})
print(data.groupby(['tag']).size())

tag
             1
bear        87
cat         95
dog         93
elephant    94
goat        80
horse       66
lion        90
tiger       92
wolf        96
dtype: int64


In [ ]:
x = data['path']
y = data['tag']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=300)

In [ ]:
X_test, X_val, y_test, y_val = train_test_split(X_train, y_train, test_size=0.7, random_state=100)

In [ ]:
# menyatukan ke dalam masing-masing dataframe
data_tr = pd.DataFrame({'path':X_train, 'tag':y_train, 'set':'training'})

data_ts = pd.DataFrame({'path':X_test, 'tag':y_test, 'set':'test'})

data_val = pd.DataFrame({'path':X_val, 'tag':y_val, 'set':'validation'})

In [ ]:
print('train size', len(data_tr))
print('val size', len(data_val))
print('test size', len(data_ts))

train size 595
val size 417
test size 178


In [ ]:
# melihat proporsi pada masing masing set apakah sudah ok atau masih ada yang ingin diubah
df_all = data_tr.append([data_val,data_ts]).reset_index(drop=1)\

print('===================================================== \n')
print(df_all.groupby(['set','tag']).size(),'\n')

print('===================================================== \n')

#cek sample datanya
df_all.sample(3)


set         tag     
test        bear        19
            cat         15
            dog         20
            elephant    28
            goat        15
            horse       28
            lion        16
            tiger       17
            wolf        20
training                 1
            bear        61
            cat         71
            dog         69
            elephant    76
            goat        66
            horse       51
            lion        61
            tiger       68
            wolf        71
validation               1
            bear        42
            cat         56
            dog         49
            elephant    48
            goat        51
            horse       23
            lion        45
            tiger       51
            wolf        51
dtype: int64 




,path,tag,set
146,/content/mammals/wolf/00000011.jpg,wolf,training
776,/content/mammals/cat/00000099.jpg,cat,validation
253,/content/mammals/cat/00000051.jpg,cat,training


In [ ]:
#Memasukkan Data pada setiap folder dari data train dan validation
dataset_path = '/content/model_mammals'

data_all = data_tr.append([data_tr, data_val, data_ts]).reset_index(drop=1)

for index, row in tq(data_all.iterrows()):
    
    #detect filepath
    file_path = row['path']
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])            
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

0it [00:00, ?it/s]

In [ ]:
os.listdir('/content/models_mammals/training')

['horse', 'elephant', 'cat', 'dog', 'tiger', 'wolf', 'bear', 'lion', 'goat']

# Augmentasi

In [ ]:
datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

In [ ]:
input_shape = (250, 250)

train = '/content/model_mammals/training'
train_generator = datagen.flow_from_directory(train, 
                                                    target_size=input_shape,
                                                    class_mode ='categorical',
                                                    batch_size = 32,
                                                    color_mode ='rgb',
                                                    shuffle = True) 

validation = '/content/model_mammals/validation'
validation_generator = datagen.flow_from_directory(validation,
                                                        target_size=input_shape,
                                                        class_mode ='categorical',
                                                        color_mode ='rgb',
                                                        batch_size = 32,
                                                        shuffle = True)

test = '/content/model_mammals/test'
test_generator = datagen.flow_from_directory(test,
                                                  target_size=input_shape,
                                                  batch_size = 32,
                                                  color_mode ='rgb',
                                                  shuffle = False)

Found 627 images belonging to 9 classes.
Found 416 images belonging to 9 classes.
Found 178 images belonging to 9 classes.
